In [6]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime
from scipy.stats import norm
from arch import arch_model
from scipy.stats import binom
import scipy.stats as stats
from numpy.random import default_rng
from scipy.stats import kstest
from numpy.random import Generator, PCG64, SeedSequence
import statsmodels.formula.api as smf
from statsmodels.regression.linear_model import RegressionResults
from scipy.stats import t
import statsmodels.api as sm
from statsmodels.tsa.api import VAR
from statsmodels.tsa.stattools import grangercausalitytests

### QUESTION 1 :
Un article récent de Welch (2022) révèle que les traders RH détiennent en fait un portefeuille
diversifié qui a, au moins depuis 2018, surperformé le marché S&P 500. Nous commencerons
notre analyse en nous faisant une idée de la performance des traders RH au cours des dernières
annéees. Téléchargez les données RH à partir du site web du cours et écrivez une fonction qui
utilisera les données sur les avoirs des utilisateurs RH pour déterminer ce que les investisseurs
RH ont détenu en moyenne au cours des dernières années.

In [7]:
df_crsp = pd.read_csv('RH_data.csv', parse_dates=['date'])

In [8]:
df_crsp

,date,users_holding,tic
0,2019-01-01,175176,AAL
1,2019-01-01,55824,AAOI
2,2019-01-01,5204232,AAPL
3,2019-01-01,125592,AAXN
4,2019-01-01,42504,ABB
...,...,...,...
436581,2020-08-13,4396732,ZNGA
436582,2020-08-13,1113509,ZOM
436583,2020-08-13,147683,ZS
436584,2020-08-13,163777,ZSAN


Dans cette étape, nous supposons que les avoirs de l'utilisateur sont les avoirs de l'investisseur représentatif pour cette action à un moment donné.

In [33]:
import pandas as pd

# Je commence par regrouper les données du fichier RH par "ticker" ou par actions : 
grouped_data = df_crsp.groupby("tic")

# Je calcule la somme totale des avoirs pour chacun ticker
total_holdings = grouped_data["users_holding"].sum()

# Je calcule les poids en divisant chaque holding par la somme des holdings à chacun des jours.
for tic, group in grouped_data:
    df_crsp.loc[group.index, "weight"] = group["users_holding"] / total_holdings[tic]


Le code ci haut permet de calculer les poids pour chaque action en divisant chaque holding par la somme des holdings de chaque jour pour chaque actions. Les poids calculés sont ensuite stockés dans la colonne "weight" du DataFrame df_crsp.

In [34]:
grouped_data = df_crsp.groupby(["date", "tic"])


In [35]:
df_crsp

,date,users_holding,tic,weight
0,2019-01-01,175176,AAL,0.000075
1,2019-01-01,55824,AAOI,0.004556
2,2019-01-01,5204232,AAPL,0.001268
3,2019-01-01,125592,AAXN,0.001583
4,2019-01-01,42504,ABB,0.003376
...,...,...,...,...
436581,2020-08-13,4396732,ZNGA,0.002339
436582,2020-08-13,1113509,ZOM,0.007685
436583,2020-08-13,147683,ZS,0.001845
436584,2020-08-13,163777,ZSAN,0.013922
